# Load to database

Rename and organize hazard, network and customer data.

Pre-requisites:
- python packages (`jupyter`, `psycopg2`, `fiona`).
- PostgreSQL database with PostGIS extension activated.
- database connection details saved to `config.json`

In [ ]:
import glob
import json
import os

import psycopg2
import fiona
import geopandas

from shapely import wkt

Load config from `config.json` (copy template from `config-template.json` and edit details).

In [ ]:
with open('config.json', 'r') as fh:
    config = json.load(fh)

Connect to the database.

In [ ]:
conn = psycopg2.connect(**config['database'])

## Create tables

Set up database tables for data loading:
- hazard outlines and scenarios
- infrastructure network nodes and edges
- boundaries and regions
- buildings

`hazard_modelled` table will contain modelled hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_modelled")
        cur.execute("""CREATE TABLE 
        hazard_modelled (
            id serial PRIMARY KEY,
            return_period int,
            model varchar(20),
            attributes jsonb,
            geom geometry(GEOMETRY,27700)
        );
        """)

`hazard_historic` table will contain historic hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_historic")
        cur.execute("""CREATE TABLE
        hazard_historic (
            id serial PRIMARY KEY,
            event varchar(20),
            start_date date,
            end_date date,
            geom geometry(GEOMETRY,27700)
        );
        """)

`infrastructure_nodes` table will contain nodes from all infrastructure sectors

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_nodes CASCADE")
        cur.execute("""CREATE TABLE
        infrastructure_nodes (
            oia_id varchar(20) PRIMARY KEY,
            source_id varchar(50),
            name varchar(60),
            sector varchar(20),
            asset_type varchar(20),
            centroid geometry(POINT,27700), -- single centroid
            catchment geometry(POLYGON,27700), -- service area/catchment
            geometry geometry(GEOMETRY,27700), -- outline or point as provided
            protected boolean,
            customers double precision,
            elec_id varchar(20) REFERENCES infrastructure_nodes (oia_id)
        );
        """)

`infrastructure_edges` table will contain physical edges from infrastructure sectors where available (e.g. road, rail)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_edges")
        cur.execute("""CREATE TABLE
        infrastructure_edges (
            oia_id varchar(40) PRIMARY KEY,
            source_id varchar(30),
            name varchar(60),
            from_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            to_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            sector varchar(20),
            geometry geometry(LINESTRING,27700), -- line as provided
            protected boolean,
            customers double precision
        );
        """)

`interdependency_edges` table will contain non-physical or inferred edges of the infrastructure network (e.g. electricity distribution)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS interdependency_edges")
        cur.execute("""CREATE TABLE
        interdependency_edges (
            oia_id varchar(40) PRIMARY KEY,
            from_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            to_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            from_sector varchar(20),
            to_sector varchar(20),
            geometry geometry(LINESTRING,27700), -- line as synthesised
            protected boolean
        );
        """)

`zones` table will contain administrative zones if/when available (e.g. national boundaries, Flood Risk Management Systems)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS zones")
        cur.execute("""CREATE TABLE
        zones (
            id serial PRIMARY KEY,
            original_id varchar(30),
            name varchar(40),
            zone_type varchar(20),
            geom geometry(GEOMETRY,27700)
        );
        """)

## Load data

Load data as provided, transforming/renaming as necessary
- modelled floods
- historic floods
- airports
- electricity cables, overhead lines, substations, towers
- gas sites, pipes
- ports
- railway lines, stations
- road links, nodes
- wastewater treatment sites
- water treatment works
- residential and non-residential buildings

### Nodes

In [ ]:
data_path = config['files']['data_path']

In [ ]:
airports = geopandas.read_file(
    os.path.join(data_path, 'airport', 'airports.shp')
).drop(
    columns=['ID', 'CLASSIFICA', 'FEATCODE', 'Name2', 'Name3', 'this_perio', 
             'rpt_apt__1', 'rpt_apt_na', 'terminal', 'transit', 'total2017']
).rename(
    columns={
      'LTIS_ID': 'source_id',
      'Name1': 'name',
      'AADF': 'customers'
    }
).reset_index()
airports['geometry'] = airports.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
airports['sector'] = 'airport'
airports['asset_type'] = 'airport'
airports['centroid'] = airports.geometry.centroid
airports['oia_id'] = airports['index'].apply(lambda n: "airp_{}".format(n))
airports.drop(columns=['index'], inplace=True)
airports['protected'] = False
airports['elec_id'] = None
airports

In [ ]:
airports.geometry[0].wkt

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'airport'
            """)
        for airport in airports.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                airport.source_id,
                airport.name,
                airport.customers,
                airport.geometry.wkb_hex, 27700,
                airport.sector,
                airport.asset_type,
                airport.centroid.wkb_hex, 27700,
                airport.oia_id,
                airport.protected,
                airport.elec_id
            ))

In [ ]:
filenames = glob.glob('../Data/Flooding data/LTISII_RFO/RecordedFloodOutlines_Final/*.shp')

In [ ]:
filenames

In [ ]:
df = geopandas.read_file(filenames[0])[
    [        
        'OUTLINE_CO',
        'NAME',
        'EVENT_CODE',
        'START_DATE',
        'END_DATE',
        'FLOOD_SRC',
        'FLOOD_CAUS',
        'FLUVIAL_IN',
        'TIDAL_IND',
        'COASTAL_IN',
        'HFM_IND',
        'geometry'
    ]
]

df.rename(columns={
    'EVENT_CODE': 'event_code',
    'OUTLINE_CO': 'id',
    'NAME': 'name',
    'START_DATE': 'start_date',
    'END_DATE': 'end_date',
    'FLOOD_SRC': 'flood_source',
    'FLOOD_CAUS': 'flood_cause',
    'FLUVIAL_IN': 'fluvial',
    'TIDAL_IND': 'tidal',
    'COASTAL_IN': 'coastal',
    'HFM_IND': 'hfm',
    'geometry': 'geom'
})

In [ ]:
df